# Exotic Option Pricing via Monte Carlo
This notebook implements a Python framework to price exotic options (barrier and basket options) using Monte Carlo simulation. All code blocks include plain-English explanations, and inputs are clearly defined as editable variables for easy scenario testing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)  # Reproducibility

# Configurable Parameters
risk_free_rate = 0.05
initial_price_A = 100.0
initial_price_B = 100.0
volatility_A = 0.20
volatility_B = 0.20
strike_price = 100.0
barrier_level = 80.0
maturity = 1.0
num_simulations = 10000
steps_for_barrier = 100

class YieldCurve:
    def __init__(self, rate):
        self.rate = rate
    def get_discount_factor(self, T):
        return np.exp(-self.rate * T)

yield_curve = YieldCurve(risk_free_rate)

In [ ]:
class Share:
    def __init__(self, name, initial_price, volatility):
        self.name = name
        self.initial_price = initial_price
        self.volatility = volatility

    def simulate_price(self, T, n_paths=1):
        r = risk_free_rate
        sigma = self.volatility
        Z = np.random.normal(size=n_paths)
        return self.initial_price * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

    def simulate_path(self, T, n_steps, n_paths=1):
        r = risk_free_rate
        sigma = self.volatility
        dt = T / n_steps
        paths = np.zeros((n_paths, n_steps + 1))
        paths[:, 0] = self.initial_price
        for step in range(1, n_steps + 1):
            Z = np.random.normal(size=n_paths)
            paths[:, step] = paths[:, step-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
        return paths

In [ ]:
class ShareOption:
    def __init__(self, underlying_share, strike, maturity, option_type="call", yield_curve=yield_curve):
        self.underlying = underlying_share
        self.strike = strike
        self.maturity = maturity
        self.option_type = option_type.lower()
        self.yield_curve = yield_curve

    def price_option(self, num_simulations=10000):
        final_prices = self.underlying.simulate_price(self.maturity, n_paths=num_simulations)
        if self.option_type == "call":
            payoffs = np.maximum(final_prices - self.strike, 0.0)
        elif self.option_type == "put":
            payoffs = np.maximum(self.strike - final_prices, 0.0)
        else:
            raise ValueError("Invalid option type.")
        return self.yield_curve.get_discount_factor(self.maturity) * np.mean(payoffs)

In [ ]:
class BarrierShareOption(ShareOption):
    def __init__(self, underlying_share, strike, maturity, barrier_level, barrier_type="down_out", option_type="call", yield_curve=yield_curve):
        super().__init__(underlying_share, strike, maturity, option_type, yield_curve)
        self.barrier_level = barrier_level
        self.barrier_type = barrier_type

    def price_option(self, num_simulations=10000, num_steps=100):
        paths = self.underlying.simulate_path(self.maturity, n_steps=num_steps, n_paths=num_simulations)
        final_prices = paths[:, -1]
        if self.barrier_type == "down_out":
            knocked_mask = (np.min(paths, axis=1) <= self.barrier_level)
        elif self.barrier_type == "up_out":
            knocked_mask = (np.max(paths, axis=1) >= self.barrier_level)
        else:
            raise ValueError("Invalid barrier type.")
        if self.option_type == "call":
            payoffs = np.maximum(final_prices - self.strike, 0.0)
        else:
            payoffs = np.maximum(self.strike - final_prices, 0.0)
        payoffs[knocked_mask] = 0.0
        return self.yield_curve.get_discount_factor(self.maturity) * np.mean(payoffs)

In [ ]:
class BasketShareOption(ShareOption):
    def __init__(self, underlying_shares, weights, strike, maturity, option_type="call", yield_curve=yield_curve):
        super().__init__(underlying_shares[0], strike, maturity, option_type, yield_curve)
        self.underlying_shares = underlying_shares
        self.weights = weights

    def price_option(self, num_simulations=10000):
        n = num_simulations
        m = len(self.underlying_shares)
        final_prices_matrix = np.zeros((n, m))
        for j, share in enumerate(self.underlying_shares):
            final_prices_matrix[:, j] = share.simulate_price(self.maturity, n_paths=n)
        basket_values = final_prices_matrix.dot(np.array(self.weights))
        if self.option_type == "call":
            payoffs = np.maximum(basket_values - self.strike, 0.0)
        else:
            payoffs = np.maximum(self.strike - basket_values, 0.0)
        return self.yield_curve.get_discount_factor(self.maturity) * np.mean(payoffs)

In [ ]:
# Create shares
stock_A = Share("Stock A", initial_price=initial_price_A, volatility=volatility_A)
stock_B = Share("Stock B", initial_price=initial_price_B, volatility=volatility_B)

# Plain European call option
plain_call = ShareOption(stock_A, strike_price, maturity, "call")
plain_price = plain_call.price_option(num_simulations)
print(f"Plain Call Option Price: {plain_price:.4f}")

# Barrier option
barrier_call = BarrierShareOption(stock_A, strike_price, maturity, barrier_level, "down_out", "call")
barrier_price = barrier_call.price_option(num_simulations, steps_for_barrier)
print(f"Barrier Call Option Price: {barrier_price:.4f}")

# Basket option
basket_call = BasketShareOption([stock_A, stock_B], [0.5, 0.5], strike_price, maturity, "call")
basket_price = basket_call.price_option(num_simulations)
print(f"Basket Call Option Price: {basket_price:.4f}")